<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/dev/ytvideo_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper

This Project uses the Original [Whisper](https://github.com/openai/whisper) Project instead of Decipher Project.

For Everyone who want to use the Original Whisper Project

# Usage

1. Check the Runtime Type is on GPU Mode in ``Runtime -> Change Runtime Type ``
1. Change the Settings at Settings Code Block
1. Click ``Runtime -> Run all`` (CTRL+F9)
1. Click on the Connect to Google Drive
1. And Wait for a moment your Generated Srt should be on ``Google Drive -> Whisper -> result``

## Add Google Drive Access 
(at Google Drive/Whisper)

In [1]:
from google.colab import drive
import os

path = '/content/gdrive/MyDrive/Whisper/'
path2 = '/content/gdrive/MyDrive/Whisper/result'
drive.mount('/content/gdrive')

if not os.path.exists(path):
  os.mkdir(path)
  os.mkdir(path2)


Mounted at /content/gdrive


# Install Components

Package : yt-dlp ffmpeg openai-whisper

In [2]:
! pip install openai-whisper yt-dlp ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 55.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 102.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 116.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796910 sha256=71e2fd020a9c43d409ff98332327ec65cf8a1cdc7646a26e9b830451064

# GPU Checkup

If Nothing shows up means you didn't enable GPU in the ``Runtime -> Change Runtime type``

In [3]:
! nvidia-smi

Mon May 29 14:45:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Settings


In [4]:
#@markdown # **Video Link Here**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=eIkK0Jx834Q" #@param {type:"string"}
# File Name Change
new_filename = "test" #@param {type:"string"}
#@markdown Note: Leave it Blank to let it audio Detect the Language
language = "" #@param {type:"string"}
task = "transcribe" #@param ['transcribe', 'translate']
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + " Transcript.txt"
# Change the Model of the Whisper
#@markdown # **Other Options**
Generate_Plain_Document = "yes" #@param ['yes','no']
shutdown_after_complete = "yes" #@param ['yes','no']
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']
enable_cookies = "no" #@param ['yes', 'no']

# If Enable Cookies is Enabled
Upload your Cookies file in here

In [6]:
import os
if enable_cookies.lower() == "yes":
  print("It is not recommended to upload your main account's cookies files here, as this is a shared environment.")
  print("Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.")
  print("The recommended approach is to use another Google Account's cookies to run this tool.")
  print("If it is a private video, you can share the video with that Google Account to grant access to it.")
  print("")

  from google.colab import files
  cookies = files.upload()

  # Specify the path where the files are located
  path_cookies = "/content"

  def rename_files(path_cookies):
      for filename in os.listdir(path_cookies):
          file_path = os.path.join(path_cookies, filename)
          if os.path.isfile(file_path) and filename.endswith(".txt"):
              new_path = os.path.join(path_cookies, "cookies.txt")
              os.rename(file_path, new_path)

  rename_files(path_cookies)

It is not recommended to upload your main account's cookies files here, as this is a shared environment.
Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.
The recommended approach is to use another Google Account's cookies to run this tool.
If it is a private video, you can share the video with that Google Account to grant access to it.



Saving blusewillstudio.txt to blusewillstudio.txt


# Download the Video
using the yt-dlp and ffmpeg

In [7]:
import yt_dlp

if enable_cookies.lower() == "yes":

  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
          'cookiefile': 'cookies.txt'
  }

else:

    ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
  }

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=eIkK0Jx834Q
[youtube] eIkK0Jx834Q: Downloading webpage
[youtube] eIkK0Jx834Q: Downloading android player API JSON
[youtube] eIkK0Jx834Q: Downloading MPD manifest
[info] eIkK0Jx834Q: Downloading 1 format(s): 251
[dashsegments] Total fragments: 3
[download] Destination: content/audio
[download] 100% of   23.07MiB in 00:00:01 at 21.23MiB/s              
[ExtractAudio] Destination: content/audio.mp3
Deleting original file content/audio (pass -k to keep)


# Generating the SRT file

In [8]:
import whisper

model = whisper.load_model(model_user)

print("Whisper model loaded.")


from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"

result = model.transcribe(input_file, verbose=True, task=task)
transcription_root = "/content/gdrive/MyDrive/Whisper/result"

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, new_filename)

if Generate_Plain_Document.lower() == "yes":

  # Save as TXT file
  srt_writer = get_writer("txt", transcription_root,)
  srt_writer(result, new_filename2)
  print("Generated srt and txt file in the Google Drive -> Whisper -> result Folder")

else:

  print("Generated srt file in the Google Drive -> Whisper -> result Folder")

100%|█████████████████████████████████████| 1.42G/1.42G [00:25<00:00, 60.4MiB/s]


Whisper model loaded.
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Chinese
[00:00.000 --> 00:02.000] 48位了
[00:02.000 --> 00:04.000] 48位了
[00:06.000 --> 00:08.000] 他的影片
[00:08.000 --> 00:10.000] 會不會破20呢
[00:12.000 --> 00:14.000] 我好奇
[00:14.000 --> 00:16.000] 所以
[00:16.000 --> 00:18.000] 我們放一個subscribe
[00:18.000 --> 00:20.000] 看我在這邊嗎
[00:26.000 --> 00:28.000] 他的loader好這是Bushwell
[00:28.000 --> 00:30.000] 我這個頻道
[00:30.000 --> 00:32.000] 我這個頻道主持在小米大大
[00:32.000 --> 00:34.000] 今天小米大大要教你的是
[00:34.000 --> 00:36.000] 如何讓影片
[00:36.000 --> 00:38.000] 上
[00:38.000 --> 00:40.000] 以下特效
[00:40.000 --> 00:42.000] 首先
[00:42.000 --> 00:44.000] 我們會先從訂閱
[00:44.000 --> 00:46.000] 特效教學
[00:46.000 --> 00:48.000] 那要怎麼上
[00:48.000 --> 00:50.000] 訂閱特效呢
[00:50.000 --> 00:52.000] 好大家可以看一下
[00:52.000 --> 00:54.000] 右上方按繼續卡
[00:54.000 --> 00:56.000] 可以跑出一個
[00:56.000 --> 00:58.000] 影片點進去
[00:58.000 --> 01:00.000] 就會來到這部影片
[01:00.000 --> 01:02.000

# Auto Shutdown the Runtime

In [9]:
if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()

Shutting Down the Runtime Because Shutdown After Complete is on
